<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [6]:
%bash
gcloud config get-value project

qwiklabs-gcp-0e6fd357ee5ebeda


In [2]:
import os
output = os.popen("gcloud config get-value project").readlines()
project_name = output[0][:-1]

# change these to try this notebook out
PROJECT = project_name
BUCKET = project_name
BUCKET = BUCKET.replace("qwiklabs-gcp-", "inna-bckt-")
REGION = 'eu-west3'

print(PROJECT)
print(BUCKET)

qwiklabs-gcp-66dff79c51c6ef7e
inna-bckt-66dff79c51c6ef7e


In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [4]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth,
  MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 100) as hashmonth_mod
FROM
  publicdata.samples.natality
WHERE year > 2000
"""
## inspect data from query:
dat_tmp = bq.Query(query + " LIMIT 100").execute().result().to_dataframe()
dat_tmp.head(n = 20)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth,hashmonth_mod
0,7.251004,False,24,1,38.0,1088037545023002395,95
1,8.750147,False,34,1,40.0,1525201076796226340,40
2,9.063204,False,27,1,38.0,411066950820961322,22
3,7.125340,False,33,1,39.0,7146494315947640619,19
4,6.759373,False,32,1,38.0,7146494315947640619,19
5,6.750554,False,20,1,39.0,7146494315947640619,19
6,7.389895,False,30,1,40.0,7146494315947640619,19
7,7.879321,False,28,1,39.0,7146494315947640619,19
8,6.686620,False,22,1,38.0,5742197815970064689,89
9,7.065815,False,25,1,41.0,6392072535155213407,7


## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [7]:
## get size of whole dataset:
query = """
SELECT COUNT(*)
FROM
  publicdata.samples.natality
WHERE year > 2000
"""
bq.Query(query).execute().result().to_dataframe()

,f0_
0,33271914


In [8]:
## calculate target sample size:
target_n_all = 15000
pop_n = 33271914
# print(pop_n / target_n_all)
print(target_n_all / pop_n)

0.0004508306916157574


In [10]:
## try out sampling (size):
query = """
SELECT COUNT(*) 
FROM 
(
  SELECT 
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE year > 2000
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) < 8           -- training
  -- AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) IN (8, 9)  -- evaluation
  AND RAND() < 0.00045083
) as tmptable
"""
bq.Query(query).execute().result().to_dataframe()

,f0_
0,12195


In [11]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE 
  year > 2000 
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) < 8
  AND RAND() < 0.00045083
"""
dat_train_raw = bq.Query(query).execute().result().to_dataframe()
print(dat_train_raw.shape)

(12255, 6)


In [15]:
dat_train_raw.head(n = 5)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.788033,False,13,1,NaN,5937540421097454372
1,5.249206,False,14,1,36.0,454960867574323744
2,7.998371,False,14,1,38.0,411066950820961322
3,6.228059,False,14,1,36.0,524531196325542205
4,7.835229,True,14,1,43.0,2363238223526193234


In [13]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE 
  year > 2000 
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) IN (8, 9)
  AND RAND() < 0.00045083
"""
dat_eval_raw = bq.Query(query).execute().result().to_dataframe()
print(dat_eval_raw.shape)

(2900, 6)


In [16]:
dat_eval_raw.head(n = 10)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.579493,False,15,1,41.0,4740473290291881219
1,6.311835,True,17,1,37.0,4740473290291881219
2,7.251004,False,18,1,40.0,4740473290291881219
3,6.499227,False,18,1,38.0,4740473290291881219
4,7.819796,False,18,1,40.0,4740473290291881219
5,1.188292,False,19,1,25.0,4740473290291881219
6,8.062305,True,19,1,39.0,4740473290291881219
7,7.431783,True,19,1,40.0,4740473290291881219
8,7.687519,True,19,1,40.0,4740473290291881219
9,8.624484,False,19,1,41.0,4740473290291881219


In [17]:
dat_train_raw.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,12242.000000,12255.000000,12255.000000,12158.000000,1.225500e+04
mean,7.242810,27.392166,1.036312,38.631765,4.371510e+18
std,1.303938,6.141916,0.198090,2.505048,2.817629e+18
min,0.599657,13.000000,1.000000,17.000000,7.493147e+16
25%,6.563162,22.000000,1.000000,38.000000,1.639186e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,6.888635e+18
max,11.750639,50.000000,4.000000,47.000000,9.183606e+18


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [18]:
dat_train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12255 entries, 0 to 12254
Data columns (total 6 columns):
weight_pounds      12242 non-null float64
is_male            12255 non-null bool
mother_age         12255 non-null int64
plurality          12255 non-null int64
gestation_weeks    12158 non-null float64
hashmonth          12255 non-null int64
dtypes: bool(1), float64(2), int64(3)
memory usage: 490.8 KB


In [19]:
dat_train_raw.isna().sum()

weight_pounds      13
is_male             0
mother_age          0
plurality           0
gestation_weeks    97
hashmonth           0
dtype: int64

In [22]:
## drop rows with missing values in training data 
## (looking forward to what will happen in the evaluation if nothing is dropped here)
dat_train = dat_train_raw.copy(deep = True).dropna()
print(dat_train_raw.shape)
print(dat_train.shape)

(12255, 6)
(12149, 6)


In [23]:
## simulate the lack of ultrasound:
dat_train_nous = dat_train.copy(deep = True)
dat_train_nous['is_male'] = 'Unknown'
dat_train_nous.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
1,5.249206,Unknown,14,1,36.0,454960867574323744
2,7.998371,Unknown,14,1,38.0,411066950820961322
3,6.228059,Unknown,14,1,36.0,524531196325542205
4,7.835229,Unknown,14,1,43.0,2363238223526193234
5,6.812284,Unknown,14,1,38.0,8599690069971956834


In [25]:
## do same preprocessing as in lab solution
## with function to preprocess both training and evaluation set:
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

dat_train = preprocess(dat_train_raw)
dat_eval = preprocess(dat_eval_raw)

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [26]:
dat_train.to_csv('train.csv', index = False, header = False)
dat_eval.to_csv('eval.csv', index = False, header = False)

In [27]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   5746 eval.csv
  24298 train.csv
  30044 total
==> eval.csv <==
7.57949256756,False,15,Single(1),41.0,4740473290291881219
6.3118345610599995,True,17,Single(1),37.0,4740473290291881219
7.25100379718,False,18,Single(1),40.0,4740473290291881219
6.4992274837599995,False,18,Single(1),38.0,4740473290291881219
7.81979643314,False,18,Single(1),40.0,4740473290291881219
1.18829159218,False,19,Single(1),25.0,4740473290291881219
8.062304921339999,True,19,Single(1),39.0,4740473290291881219
7.4317828520199996,True,19,Single(1),40.0,4740473290291881219
7.68751907594,True,19,Single(1),40.0,4740473290291881219
8.62448368944,False,19,Single(1),41.0,4740473290291881219

==> train.csv <==
5.24920645822,False,14,Single(1),36.0,454960867574323744
7.99837086536,False,14,Single(1),38.0,411066950820961322
6.2280589015,False,14,Single(1),36.0,524531196325542205
7.83522879148,True,14,Single(1),43.0,2363238223526193234
6.8122838958,False,14,Single(1),38.0,8599690069971956834
6.87621795178,True,14,Single(1),39.0

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License